In [16]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import model_test_for_openmax_add_CDF

ModuleNotFoundError: No module named 'model_test_for_openmax_add_CDF'

In [17]:
input_data = input()         # 아직 입력부를 구현하지 않아 임시로 입력

In [ ]:
def tokenize(x_data, tokenizer):
    tokenizer.fit_on_texts(x_data) # 데이터의 각 행별로 토큰화 수행
    return tokenizer.texts_to_sequences(x_data)    

In [ ]:
x_data = input_data

In [ ]:
tokenizer = Tokenizer()
sequences = tokenize(x_data, tokenizer)

## 모델 적용

In [ ]:
from.models import model_from_json_file = open("model.json", "r")
loaded_model_json = json_file.road() json_file.close()
loaded_model = model_from_json(loaded_model_json)

In [ ]:
loaded_model.loade_weights("model_weights.h5")

In [ ]:
loaded_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=["accuracy",f1,precision, recall])

## logit Vector, Distance 계산

In [ ]:
class_result = 0
class_result.append(numbers.index(max(X_result)))

tf_result = (y_test == class_result)

for_open_max_0 = []; 
for_open_max_1 = [];
for_open_max_2 = [];

if(tf_result == True):
    if(class_result == 0):
        for_open_max_0.append(X_result)
    if(class_result == 1):
        for_open_max_1.append(X_result)
    if(class_result == 2):
        for_open_max_2.append(X_result)

In [ ]:
input_average_0 = average_vector(for_open_max_0, class_result) #평균 Logit Vector - Class 0
input_average_1 = average_vector(for_open_max_1, class_result) #평균 Logit Vector - Class 1
input_average_2 = average_vector(for_open_max_2, class_result) #평균 Logit Vector - Class 2

In [ ]:
dist0 = distance(for_open_max_0, input_average_0)
dist1 = distance(for_open_max_1, input_average_1)
dist2 = distance(for_open_max_2, input_average_2)

## CDF 계산

In [ ]:
def calculCDF(dist):
    CDF = []
    for i in dist:
        CDF.append(s.exponweib.cdf(i, *s.exponweib.fit(i, 1, 1, scale=2, loc=0)))     
        return CDF
CDF0 = calculCDF(dist0)
CDF1 = calculCDF(dist1)
CDF2 = calculCDF(dist2)

## Logit Vetcr 업데이트

In [ ]:
updated_logit = []
i = 0

logit0 = X_result[0]-(X_result[0]*CDF0[i])
logit1 = X_result[1]-(X_result[1]*CDF1[i])
logit2 = X_result[2]-(X_result[2]*CDF2[i])
unkn_logit = CDF0[i]*X_result[0] + CDF1[i]*X_result[1] + CDF2[i]*X_result[2]    # unknown class의 logit vector
updated_logit.append([unkn_logit, logit0, logit1, logit2])     
i += 1

In [ ]:
# update 된 logit veoctor로 softmax layer 통과